In [1]:
import numpy as np
import pickle
import h5py
from tqdm import tqdm

## Load the objects with the crystal structure and supercell data

In [2]:
NNList = np.load("NNsites_sitewise.npy")
GpermNNIdx = np.load("GroupNNpermutations.npy")
dxList = np.load("dxList.npy")
jList = np.load("jList.npy")
dxR = np.load("dxR.npy")
jumpSiteIndex = np.load("JumpNewSiteIndices.npy")

RtoSiteInd = np.load("RtoSiteInd.npy")
siteIndtoR = np.load("SiteIndtoR.npy")

with open("GroupOpsIndices.pkl", "rb") as fl:
    GIndtoGDict = pickle.load(fl)

with open("supercellFCC.pkl", "rb") as fl:
    superFCC = pickle.load(fl)

with open("jnetFCC.pkl", "rb") as fl:
    jnetFCC = pickle.load(fl)

In [3]:
dxList # contains the jump vectors.

array([[ 0. , -0.5,  0.5],
       [-0. ,  0.5, -0.5],
       [ 0.5,  0. , -0.5],
       [-0.5, -0. ,  0.5],
       [ 0. , -0.5, -0.5],
       [-0. ,  0.5,  0.5],
       [-0.5, -0.5,  0. ],
       [ 0.5,  0.5, -0. ],
       [ 0.5,  0. ,  0.5],
       [-0.5, -0. , -0.5],
       [ 0.5, -0.5,  0. ],
       [-0.5,  0.5, -0. ]])

In [4]:
# Convert to lattice vectors.
dxNNVecs = np.zeros_like(dxList).astype(int)
for jmp in range(dxList.shape[0]):
    dx = dxList[jmp]
    dxNN, _ = superFCC.crys.cart2pos(dx)
    dxNNVecs[jmp, :] = dxNN[:]
    
assert np.array_equal(dxNNVecs, dxR)

## Verify Nearest neighbor sites under saved order of jumps

In [6]:
for siteInd in tqdm(range(NNList.shape[1]), position=0, leave=True, ncols=65):
    assert NNList[0, siteInd] == siteInd
    ciSite, Rsite = superFCC.ciR(siteInd)
    for jmp in range(NNList.shape[0] - 1):
        RsiteNew = Rsite + dxR[jmp]
        siteNew, _ = superFCC.index(RsiteNew, (ciSite))
        assert NNList[jmp + 1, siteInd] == siteNew
    
        if siteInd == 0:
            assert jList[jmp] == siteNew

100%|████████████████████████| 512/512 [00:00<00:00, 4787.55it/s]


## Now test the group permutation of nearest neigbhors

In [15]:
considered = set()
for gInd, g in tqdm(GIndtoGDict.items(), position=0, leave=True, ncols=65):
    assert g in superFCC.crys.G
    assert g not in considered
    considered.add(g)
    for jmp in range(dxList.shape[0]):
        jmpvec = dxList[jmp]
        jmpvecRot = superFCC.crys.g_cart(g, jmpvec)
        
        idxnew = None
        count = 0
        for jmpNew in range(dxList.shape[0]):
            if np.allclose(dxList[jmpNew], jmpvecRot):
                count += 1
                idxnew = jmpNew
        assert count == 1
        assert GpermNNIdx[gInd, idxnew + 1] == jmp + 1

assert len(GIndtoGDict) == 48

100%|███████████████████████████| 48/48 [00:00<00:00, 123.63it/s]


## Now check re-indexing of sites after jumps

In [11]:
# Now for each jump, displace periodically and check
# Now test the nearest neighbor indexing
randomState = np.random.randint(1, 5, NNList.shape[1], dtype=np.int8)
print(randomState.shape)
# Put a vacancy at 0
randomState[0] = 0


for jmp in tqdm(range(dxList.shape[0]), position=0, leave=True, ncols=65):
    
    dxRVec, _ = superFCC.crys.cart2pos(dxList[jmp])
    
    state2 = randomState.copy()
    assert state2[0] == 0 # check initially there was vacancy
    state2[0] = state2[jList[jmp]]
    state2[jList[jmp]] = 0
    state2Trans = np.zeros_like(state2)
    for site in range(NNList.shape[1]):
        ciSite, Rsite = superFCC.ciR(site)
        RsiteNew = Rsite - dxRVec # translate by negative of vac jump
        siteIndNew = superFCC.index(RsiteNew, ciSite)
        state2Trans[siteIndNew] = state2[site]
    
    assert state2Trans[0] == 0
    assert np.array_equal(randomState[jumpSiteIndex[jmp]], state2Trans)

(512,)


100%|████████████████████████████| 12/12 [00:00<00:00, 84.07it/s]


## Check the site index arrays

In [13]:
for siteInd in range(siteIndtoR.shape[0]):
    R = siteIndtoR[siteInd]
    siteIndCalc = R[0] * 8 * 8 + R[1] * 8 + R[2]
    assert siteIndCalc == siteInd
    assert RtoSiteInd[R[0], R[1], R[2]] == siteInd
    siteIndSuperCell , _ = superFCC.index(R, (0,0))
    assert siteIndCalc == siteIndSuperCell

## Now test the 2nn arrays

In [ ]:
GpermNNIdx_2nn = np.load("GroupNNpermutations_2nn.npy")
dxR_2nn = np.load("nnJumpLatVecs_2nn.npy")
NNsites_2nn = np.load("NNsites_sitewise_2nn.npy")

## First, the group permutations of the nearest neighbors

In [23]:
considered = set()
for gInd, g in tqdm(GIndtoGDict.items(), position=0, leave=True, ncols=65):
    considered.add(g)
    for RnnInd in range(dxR_2nn.shape[0]):
        Rnn = dxR_2nn[RnnInd]
        RnnRot, _ = superFCC.crys.g_pos(g, Rnn, (0, 0))
        idxnew = None
        count = 0
        for RnnInd_New in range(dxR_2nn.shape[0]):
            if np.allclose(dxR_2nn[RnnInd_New], RnnRot):
                count += 1
                idxnew = RnnInd_New
        assert count == 1
        assert GpermNNIdx_2nn[gInd, idxnew + 1] == RnnInd + 1

100%|████████████████████████████| 48/48 [00:01<00:00, 46.01it/s]


## Then the nearest neighbors

In [24]:
for siteInd in tqdm(range(NNsites_2nn.shape[1]), position=0, leave=True, ncols=65):
    assert NNsites_2nn[0, siteInd] == siteInd
    ciSite, Rsite = superFCC.ciR(siteInd)
    for jmp in range(NNsites_2nn.shape[0] - 1):
        RsiteNew = Rsite + dxR_2nn[jmp]
        siteNew, _ = superFCC.index(RsiteNew, (ciSite))
        assert NNsites_2nn[jmp + 1, siteInd] == siteNew
    
        if siteInd == 0 and jmp < dxList.shape[0]:
            assert jList[jmp] == siteNew == NNList[jmp+1, 0]

100%|████████████████████████| 512/512 [00:00<00:00, 3363.82it/s]
